### 1. Selección del dataset:
  * Utiliza el dataset que seleccionaste para el Proyecto 2 o elige uno adecuado de Kaggle que se relacione con tu área de interés. El dataset debe tener al menos 10 características y una variable objetivo clara.
  * Realiza una exploración inicial para entender las variables y la relación que tienen con la variable objetivo.

### 2. Reducción de dimensionalidad:
  * Implementa un análisis de reducción de dimensionalidad utilizando PCA (u otra técnica adecuada). El objetivo es reducir el número de características, manteniendo al menos el 80-90% de la variabilidad original.
  * Visualiza los componentes principales y discute cómo contribuyen a la variabilidad del dataset.
  * Utiliza las características reducidas para simplificar el conjunto de datos.

### 3. Benchmarking de modelos:
  * Implementa y compara al menos tres modelos de machine learning en el dataset reducido. Algunos ejemplos de modelos que puedes utilizar incluyen:
    * Regresión logística
    * Árboles de decisión
    * Random Forest
    * Support Vector Machines (SVM)
  * Para cada modelo, utiliza validación cruzada para medir su rendimiento de manera consistente. Registra las métricas más importantes, como la precisión, el recall, la F1-score y el AUC-ROC.

### 4. Análisis comparativo de modelos:
  * Compara los modelos en base a las métricas obtenidas. Discute cuál de los modelos es más adecuado para resolver el problema en cuestión y por qué.
  * Si alguno de los modelos tiene un rendimiento significativamente inferior, analiza posibles causas y estrategias para mejorar el rendimiento (como ajustar hiperparámetros o utilizar técnicas de regularización).

### 5. Informe final:
  * Elabora un informe detallado que incluya:
    * El análisis de reducción de dimensionalidad, mostrando cómo PCA (u otra técnica) ayudó a reducir el número de características sin perder información relevante.
    * Un análisis comparativo de los modelos probados, con gráficos que muestren el rendimiento de cada modelo.
    * Conclusiones y recomendaciones sobre cuál modelo utilizarías en un escenario real.